### Using the awk commands for parsing the log file

In [1]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

from shutil import rmtree

root_path = os.path.join(Path().resolve().parent.parent)
package_path = os.path.join(root_path, "hamageolib")
test_fixture_path = os.path.join(root_path, "big_tests", "TwoDSubduction", "eba_cdpt_coh500_SA80.0_cd7.5_log_ss300.0")
result_path = os.path.join(Path().resolve(), "results")
output_dir = os.path.join(result_path, "case_options_output")

if str(package_path) not in sys.path:
    sys.path.insert(0, str(package_path))

if not os.path.isdir(result_path):
    os.mkdir(result_path)

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

from utils.case_options import parse_log_file_for_time_info_to_pd


#### Parsing Log File for Time Step and Wall Clock Information

This block extracts and processes time step and wall clock data from an ASPECT log file.

- Define the path to the log file: $ \text{log\_file\_path} $ within the test fixture directory.
- Define the output file path: $ \text{output\_path} $ for storing parsed results.
- Use the function `parse_log_file_for_time_info_to_pd()` to extract time-related information into a pandas DataFrame.
- The resulting pandas data structure contains a "Corrected Wall Clock" as output for wall clocks corrected for restarting the case.

In [2]:
log_file_path = os.path.join(test_fixture_path, "output", "log.txt")
output_path = os.path.join(result_path, "parse_time_info_results")

time_dat = parse_log_file_for_time_info_to_pd(log_file_path)

#### Use the CASE_OPTIONS class to load a case

This class read the parameter files from aspect and related files from WorldBuilder.
It stores the useful options for further analysis (uncomment the print options below to output).

- case_dir (str): Directory path of the case.
- output_dir (str): Directory path for output files.
- visit_file (str): Path to the visit file for case visualization.
- paraview_file (str): Path to the paraview file for case visualization.
- img_dir (str): Directory path for image outputs.
- idict (dict): Dictionary containing parsed parameters from the .prm file.
- wb_dict (dict): Dictionary containing parsed parameters from the .wb file if it exists.
- options (dict): Dictionary storing interpreted options for data output and model parameters.

More specifically, some small files from ASPECT are already imported into handy format

- statistic_df: a pandas object contains the data from the "output/statistics"
- time_df: a pandas object with parsed data from the "output/log.txt" contains the time step, model time, and wallclock time.

In [3]:
from utils.case_options import CASE_OPTIONS

Case_Options = CASE_OPTIONS(test_fixture_path)

# print(Case_Options.__dict__)
# print(Case_Options.statistic_df)
# print(Case_Options.time_df)
# print(Case_Options.visualization_df)

#### Generate a summary of the current case

Note: first, the class needs to be interpreted.

In [4]:
Case_Options.Interpret()
Case_Options.SummaryCaseVtuStep()
print(Case_Options.summary_df)

     Vtu step          Time  Time step number  Vtu snapshot  File found
4           0  0.000000e+00                 0             4        True
5           1  1.029984e+05                27             5        True
6           2  2.024379e+05                46             6        True
7           3  3.065885e+05                62             7        True
8           4  4.017505e+05                75             8        True
..        ...           ...               ...           ...         ...
96         92  9.206712e+06               969            96        True
97         93  9.300098e+06               980            97        True
98         94  9.404288e+06               996            98        True
99         95  9.501962e+06              1022            99        True
100        96  9.600502e+06              1062           100        True

[97 rows x 5 columns]


### Use the CASE_SUMMARY class to generate summary of a project

This code loops through **case directories** inside `"big_tests/TwoDSubduction"` and updates the case summary. 

#### **To locate a case:**
- Each valid case directory **must contain a `.prm` file**.
- The `.prm` file is identified using `find_case_files(item_path)`.
- If no `.prm` file is found, the directory is skipped.
- The summary stores essential metadata about cases.

#### **Output:**
- The script updates the df pandas object as an attribute of the class
- For example, `Case_Summary.df["basename"]` stores processed case names.
- The `Case_Summary.df.columns` stores all the available names.
- The `Case_Summary.df.attrs["units"]` stores all the units.
- The final output prints the collected case information.

In [ ]:
from utils.case_options import CASE_SUMMARY, find_case_files

# Initiate the class
Case_Summary = CASE_SUMMARY()

# Assign a directory to loop over
test_fixture_parent_dir = os.path.join(root_path, "big_tests", "TwoDSubduction")

# find case directory and update summary
for item in os.listdir(test_fixture_parent_dir):
    item_path = os.path.join(test_fixture_parent_dir, item)

    if os.path.isdir(item_path):
        prm_file, _ = find_case_files(item_path)
        
        if prm_file is not None:
            Case_Summary.update_single_case(item_path)

file_path = os.path.join(output_dir, "case_summary.csv")
Case_Summary.df.to_csv(file_path)
print("Saved file %s" % (file_path))

# print(str(Case_Summary.df))
# print(str(Case_Summary.df.attrs["units"]))

Saved file /home/lochy/ASPECT_PROJECT/HaMaGeoLib/notebooks/examples/results/case_options_output/case_summary.csv
                                          basename  name end time step  \
0                                       test_visit  None             5   
1  eba_cdpt_coh500_SA80.0_OA40.0_cd100.0_cd7.5_gr9  None          3337   
2                 EBA_CDPT_test_perplex_mixing_log  None          4040   
3         eba_cdpt_coh500_SA80.0_cd7.5_log_ss300.0  None          1170   
4                             test_get_snaps_steps  None           0.0   
5                               test_visit_default  None             0   

     end time  wall clock                                           abs path  
0   1000000.0        21.6  /home/lochy/ASPECT_PROJECT/HaMaGeoLib/big_test...  
1  60000000.0    653000.0  /home/lochy/ASPECT_PROJECT/HaMaGeoLib/big_test...  
2  60000000.0    968000.0  /home/lochy/ASPECT_PROJECT/HaMaGeoLib/big_test...  
3   9627890.0    526000.0  /home/lochy/ASPECT_PROJEC

/home/lochy/ASPECT_PROJECT/HaMaGeoLib/hamageolib/utils/case_options.py:414: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)


#### Convert to a latex table

- A `column_names` could be added to select the columns to export. Otherwise all columns are exported.

In [6]:
from utils.case_options import df_to_latex

output_path = os.path.join(result_path, "case_options_results.tex")

columnn_names = ["name", "end time step", "end time"]

df_to_latex(Case_Summary.df, output_path, columnn_names)

'LaTeX table saved to /home/lochy/ASPECT_PROJECT/HaMaGeoLib/notebooks/examples/results/case_options_results.tex'